In [1]:
import torch
import torch.nn as nn

import numpy as np
from scipy.io import loadmat, savemat
import math
import os
import h5py

from functools import partial
from models.models import MWT1d
from models.utils import train, test, LpLoss, get_filter, UnitGaussianNormalizer

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def get_initializer(name):
    
    if name == 'xavier_normal':
        init_ = partial(nn.init.xavier_normal_)
    elif name == 'kaiming_uniform':
        init_ = partial(nn.init.kaiming_uniform_)
    elif name == 'kaiming_normal':
        init_ = partial(nn.init.kaiming_normal_)
    return init_

In [6]:
ntrain = 1000
ntest = 200

sub = 2**3 #subsampling rate
h = 2**13 // sub #total grid size divided by the subsampling rate
s = h
batch_size = 20

rw_ = loadmat('Data/KDV/kdv_train_test.mat')
x_data = rw_['input'].astype(np.float32)
y_data = rw_['output'].astype(np.float32)

In [7]:
x_train = x_data[:ntrain,::sub]
y_train = y_data[:ntrain,::sub]
x_test = x_data[-ntest:,::sub]
y_test = y_data[-ntest:,::sub]

x_train = torch.from_numpy(x_train)
x_test = torch.from_numpy(x_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

x_train = x_train.unsqueeze(-1)
x_test = x_test.unsqueeze(-1)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

In [8]:
ich = 1
initializer = get_initializer('xavier_normal') # xavier_normal, kaiming_normal, kaiming_uniform

model = MWT1d(ich,
            alpha = 10,
            c = 4*4,
            k = 4,
            base = 'legendre', # chebyshev
            nCZ = 2,
            initializer = initializer,
            ).to(device)
learning_rate = 0.001

epochs = 500
step_size = 100
gamma = 0.5

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)

for epoch in range(1, epochs+1):
    train_l2 = train(model, train_loader, optimizer, epoch, device,
        lossFn = myloss, lr_schedule = scheduler)
    
    test_l2 = test(model, test_loader, device, lossFn=myloss)
    print(f'epoch: {epoch}, train l2 = {train_l2}, test l2 = {test_l2}')